In [1]:
import os
import os.path as osp

from PIL import Image
import numpy as np
import torch

from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

import motmetrics as mm
mm.lap.default_solver = 'lap'

import torchvision
import yaml
from tqdm import tqdm
import sacred
from sacred import Experiment
import motmetrics as mm
import sys
sys.path.append("..")
from src.tracktor.utils import interpolate, plot_sequence, get_mot_accum, evaluate_mot_accums
from dataset.eval_track import COSMOSTestDataset, get_mot_accum_new, get_mot_accum_new_2

# Evaluation for Vehicle
### Read Ground Truth Dataset

In [2]:
dataset = []
dataset_path = '../../COSMOS_data_tracktor/Track_data_cosmos/only_v'
for video_root in os.listdir(dataset_path):
    if 'sec' in video_root:
        # print(osp.join(tracktor['dataset2'], video_root))
        dataset.append(COSMOSTestDataset(osp.join(dataset_path, video_root)))

In [3]:
for seq in dataset:
    print(str(seq))

traffic_video_GP020614_190720_0237_0407_90sec_calibrated
traffic_video_GP020587_190720_0900_1030_90sec_calibrated
traffic_video_GP010615_190724_0545_0715_90sec_calibrated
traffic_video_HIKL1D190911T153514_20190920_0700_0830_90sec_calibration
traffic_video_HIKL1D190821T200329_190916_1715_1845_90sec_calied
traffic_video_GP010589_190720_0310_0440_90sec_calibrated
traffic_video_GOPR0589_190720_1324_1454_90sec_calibrated
traffic_video_HIKL1D190821T200329_190919_2350_2520_90sec_calied
traffic_video_HIKL1D190821T000900_190920_0152_0322_90sec_calied


### Change result .txt file to dirct

In [4]:
results_path = '../output/tracktor/COSMOS/Tracktor++'
results_files = []
for seq in dataset:
    results_files.append(osp.join(results_path, str(seq)+'_v.txt'))

In [5]:
results_files

['../output/tracktor/COSMOS/Tracktor++/traffic_video_GP020614_190720_0237_0407_90sec_calibrated_v.txt',
 '../output/tracktor/COSMOS/Tracktor++/traffic_video_GP020587_190720_0900_1030_90sec_calibrated_v.txt',
 '../output/tracktor/COSMOS/Tracktor++/traffic_video_GP010615_190724_0545_0715_90sec_calibrated_v.txt',
 '../output/tracktor/COSMOS/Tracktor++/traffic_video_HIKL1D190911T153514_20190920_0700_0830_90sec_calibration_v.txt',
 '../output/tracktor/COSMOS/Tracktor++/traffic_video_HIKL1D190821T200329_190916_1715_1845_90sec_calied_v.txt',
 '../output/tracktor/COSMOS/Tracktor++/traffic_video_GP010589_190720_0310_0440_90sec_calibrated_v.txt',
 '../output/tracktor/COSMOS/Tracktor++/traffic_video_GOPR0589_190720_1324_1454_90sec_calibrated_v.txt',
 '../output/tracktor/COSMOS/Tracktor++/traffic_video_HIKL1D190821T200329_190919_2350_2520_90sec_calied_v.txt',
 '../output/tracktor/COSMOS/Tracktor++/traffic_video_HIKL1D190821T000900_190920_0152_0322_90sec_calied_v.txt']

In [6]:
all_result = {}
for i, seq in enumerate(dataset):
    results = {}
    results_file = results_files[i]
    with open(results_file,'r') as file:
        data = file.readlines()
        for line in data:
            l = line.split(',')
            obj_id = int(l[1]) - 1
            frame = int(l[0]) - 1
            if obj_id not in results:
                results[obj_id] = {}
            results[obj_id][frame] = np.array([float(l[2]), float(l[3]), 
                                          float(l[2]) + float(l[4]),
                                          float(l[3]) + float(l[5])])
    all_result[str(seq)] = results 

### Calculate MOT Metrics

In [7]:
final_MOT = []
for seq in dataset:
    mot_accums = []
    results = all_result[str(seq)]
    mot_accum = get_mot_accum_new(results, seq)
    mot_accums.append(mot_accum)
    
    res = evaluate_mot_accums(mot_accums, [str(seq)])
    final_MOT.append(res)

                                                          IDF1   IDP   IDR  Rcll  Prcn GT MT PT ML  FP FN IDs  FM  MOTA  MOTP
traffic_video_GP020614_190720_0237_0407_90sec_calibrated 98.3% 97.0% 99.6% 99.6% 97.0% 34 34  0  0 297 35   1   1 96.5% 0.230
                                                          IDF1   IDP   IDR  Rcll  Prcn GT MT PT ML   FP   FN IDs   FM  MOTA  MOTP
traffic_video_GP020587_190720_0900_1030_90sec_calibrated 85.2% 82.9% 87.6% 89.1% 84.3% 66 61  2  3 5103 3347  57  346 72.4% 0.218
                                                          IDF1   IDP   IDR  Rcll  Prcn GT MT PT ML   FP  FN IDs  FM  MOTA  MOTP
traffic_video_GP010615_190724_0545_0715_90sec_calibrated 93.0% 88.6% 97.8% 99.3% 89.9% 60 59  0  1 2078 136   1  14 88.2% 0.189
                                                                        IDF1   IDP   IDR  Rcll  Prcn GT MT PT ML   FP  FN IDs  FM  MOTA  MOTP
traffic_video_HIKL1D190911T153514_20190920_0700_0830_90sec_calibration 90.4% 86.1% 95.1% 9

# Evaluation for pedestrian
### Read Ground Truth Dataset

In [8]:
dataset = []
dataset_path = '../../COSMOS_data_tracktor/Track_data_cosmos/only_p'
for video_root in os.listdir(dataset_path):
    if 'sec' in video_root:
        # print(osp.join(tracktor['dataset2'], video_root))
        dataset.append(COSMOSTestDataset(osp.join(dataset_path, video_root)))

In [9]:
for seq in dataset:
    print(str(seq))

traffic_video_GP020614_190720_0237_0407_90sec_calibrated
traffic_video_GP020587_190720_0900_1030_90sec_calibrated
traffic_video_GP010615_190724_0545_0715_90sec_calibrated
traffic_video_HIKL1D190911T153514_20190920_0700_0830_90sec_calibration
traffic_video_HIKL1D190821T200329_190916_1715_1845_90sec_calied
traffic_video_GP010589_190720_0310_0440_90sec_calibrated
traffic_video_GOPR0589_190720_1324_1454_90sec_calibrated
traffic_video_HIKL1D190821T200329_190919_2350_2520_90sec_calied
traffic_video_HIKL1D190821T000900_190920_0152_0322_90sec_calied


In [10]:
results_path = '../output/tracktor/COSMOS/Tracktor++'
results_files = []
for seq in dataset:
    results_files.append(osp.join(results_path, str(seq)+'_p.txt'))

In [11]:
results_files

['../output/tracktor/COSMOS/Tracktor++/traffic_video_GP020614_190720_0237_0407_90sec_calibrated_p.txt',
 '../output/tracktor/COSMOS/Tracktor++/traffic_video_GP020587_190720_0900_1030_90sec_calibrated_p.txt',
 '../output/tracktor/COSMOS/Tracktor++/traffic_video_GP010615_190724_0545_0715_90sec_calibrated_p.txt',
 '../output/tracktor/COSMOS/Tracktor++/traffic_video_HIKL1D190911T153514_20190920_0700_0830_90sec_calibration_p.txt',
 '../output/tracktor/COSMOS/Tracktor++/traffic_video_HIKL1D190821T200329_190916_1715_1845_90sec_calied_p.txt',
 '../output/tracktor/COSMOS/Tracktor++/traffic_video_GP010589_190720_0310_0440_90sec_calibrated_p.txt',
 '../output/tracktor/COSMOS/Tracktor++/traffic_video_GOPR0589_190720_1324_1454_90sec_calibrated_p.txt',
 '../output/tracktor/COSMOS/Tracktor++/traffic_video_HIKL1D190821T200329_190919_2350_2520_90sec_calied_p.txt',
 '../output/tracktor/COSMOS/Tracktor++/traffic_video_HIKL1D190821T000900_190920_0152_0322_90sec_calied_p.txt']

In [12]:
all_result = {}
for i, seq in enumerate(dataset):
    results = {}
    results_file = results_files[i]
    with open(results_file,'r') as file:
        data = file.readlines()
        for line in data:
            l = line.split(',')
            obj_id = int(l[1]) - 1
            frame = int(l[0]) - 1
            if obj_id not in results:
                results[obj_id] = {}
            results[obj_id][frame] = np.array([float(l[2]), float(l[3]), 
                                          float(l[2]) + float(l[4]),
                                          float(l[3]) + float(l[5])])
    all_result[str(seq)] = results 

In [13]:
final_MOT = []
for seq in dataset:
    mot_accums = []
    results = all_result[str(seq)]
    mot_accum = get_mot_accum_new_2(results, seq)
    mot_accums.append(mot_accum)
    
    res = evaluate_mot_accums(mot_accums, [str(seq)])
    final_MOT.append(res)

                                                          IDF1   IDP   IDR  Rcll  Prcn GT MT PT ML   FP   FN IDs   FM  MOTA  MOTP
traffic_video_GP020614_190720_0237_0407_90sec_calibrated 66.1% 65.4% 66.9% 73.5% 71.8% 38 22 14  2 1890 1736  49  262 43.9% 0.360
                                                          IDF1   IDP   IDR  Rcll  Prcn GT MT PT ML   FP   FN IDs   FM  MOTA  MOTP
traffic_video_GP020587_190720_0900_1030_90sec_calibrated 55.8% 57.4% 54.3% 66.3% 70.2% 38 13 22  3 2969 3549  49  527 37.7% 0.388
                                                          IDF1   IDP   IDR  Rcll  Prcn GT MT PT ML   FP   FN IDs   FM  MOTA  MOTP
traffic_video_GP010615_190724_0545_0715_90sec_calibrated 63.0% 64.4% 61.6% 74.9% 78.3% 84 44 31  9 3605 4380 190  716 53.1% 0.325
                                                                        IDF1   IDP   IDR  Rcll  Prcn GT MT PT ML   FP   FN IDs   FM   MOTA  MOTP
traffic_video_HIKL1D190911T153514_20190920_0700_0830_90sec_calibration 42.3